In [1]:
import pandas as pd
import ast
from datetime import time, datetime, timedelta
import numpy as np

In [20]:
def __to_hr_mean(x):
    try:
        return np.mean(1000/np.array(ast.literal_eval(x))*60)
    except:
        return 0

def nni_to_hr_means(df):
    df['hr_means'] = df['nni'].apply(lambda x: __to_hr_mean(x))
    return df

def load_csv_file(csv_file):
    df = pd.read_csv(csv_file)    
    # convert to datetime
    df['from_time'] = pd.to_datetime(df['from_time'], format='%Y-%m-%d %H:%M:%S.%f')
    df['to_time'] = pd.to_datetime(df['to_time'], format='%Y-%m-%d %H:%M:%S.%f')

    return df

def convert_to_awd(df, column, awd_file, data_meta):
    df[column].to_csv(awd_file, index=False)
    with open(awd_file, "r") as f:
        contents = f.readlines()

    for i, v in enumerate(data_meta.values()):
        contents.insert(i, v+'\n')

    with open(awd_file, "w") as f:
        contents = "".join(contents)
        f.write(contents)

def generate_awd_from_calculated_nni_df(csv_file, channels, duration, label=''):
    # load dataframe
    df = load_csv_file(csv_file)
    
    data_log = {}
    
    # loop for each channel
    for channel in channels:
        data_meta = {
            'name' : f'{label}-Channel-{channel}',
            'date' : min(df['from_time']).date().strftime("%d-%m-%Y"),
            'time' : min(df['from_time']).strftime("%H:%M"),
            'duration' : str(duration*4), # duration in minutes * 4
            'age' : '0',
            'id' : f'{label}-Channel-{channel}',
            'sex' : 'N'
        }
        
        df_hrv = df[df.channel==channel]
        
        df_hrv = nni_to_hr_means(df_hrv)
        
        # remove duplicate
        df_hrv = df_hrv.drop_duplicates(subset='from_time', keep='first')

        # sort by from_time
        df_hrv = df_hrv.sort_values(by='from_time')

        records_per_day  = int(60 * 24 / duration)

        start_date = datetime.combine(min(df_hrv.from_time).date(), datetime.min.time())
        finish_date = datetime.combine(max(df_hrv.from_time).date(), datetime.min.time())
        total_days = (finish_date - start_date).days + 1

        data_log[channel] = {}

        df_hr_filter = pd.DataFrame(columns=df_hrv.columns)
        for day_num in range(total_days):
            current_date = start_date + timedelta(days=day_num)
            prev_date = current_date - timedelta(days=1)
            next_date = current_date + timedelta(days=1)
            df_records = df_hrv[df_hrv.from_time.gt(current_date) & df_hrv.from_time.le(next_date)]
            data_log[channel][current_date] = df_records.shape[0]
            if current_date == start_date:
                pass
            elif current_date == finish_date:
                pass
            else: # need to check
                if df_records.shape[0] < records_per_day:
                    data_add = [list(df_hrv.columns)]
                    for i in range(records_per_day):
                        check_0 = current_date + timedelta(minutes=duration*i)
                        check_1 =  current_date + timedelta(minutes=duration*(i+1))
                        p_check_0 = prev_date + timedelta(minutes=duration*i)
                        p_check_1 =  prev_date + timedelta(minutes=duration*(i+1))
                        n_check_0 = next_date + timedelta(minutes=duration*i)
                        n_check_1 =  next_date + timedelta(minutes=duration*(i+1))
                        available = df_records[df_records.from_time.gt(check_0) & df_records.from_time.le(check_1)]
                        if len(available) == 0:
                            # get previous day
                            p_available = df_hrv[df_hrv.from_time.gt(p_check_0) & df_hrv.from_time.le(p_check_1)]
                            # get next day
                            n_available = df_hrv[df_hrv.from_time.gt(n_check_0) & df_hrv.from_time.le(n_check_1)]
                            # now compare
                            if len(p_available)+len(n_available) > 1:
                                data_add += [[0, 0, (p_available.activity.iloc[0]+n_available.activity.iloc[0])/2,0, check_0, check_1, channel, (p_available.hr_means.iloc[0]+n_available.hr_means.iloc[0])/2]]
                            elif len(p_available) > 0:
                                added = [val for val in p_available.iloc[0]]
                                added[4] = check_0
                                added[5] = check_1
                                data_add += [added]
                            elif len(n_available) > 0:
                                added = [val for val in n_available.iloc[0]]
                                added[4] = check_0
                                added[5] = check_1
                                data_add += [added]
                            else:
                                data_add += [[0, 0, 0, 0, check_0, check_1, channel, 0]]
                    if len(data_add)>1:
                        #define second DataFrame
                        data_add = {data[0]:data[1:]for data in list(zip(*data_add))}
                        df_add = pd.DataFrame(data_add)
                        #add new row to end of DataFrame
                        df_records = pd.concat([df_records, df_add], ignore_index=True)

                if df_records.shape[0] > records_per_day:
                    df_records = df_records[:records_per_day]

            df_hr_filter = pd.concat([df_hr_filter, df_records], ignore_index=True)
        
#         # interpolate empty data
#         df_hr_filter['hr_means'] = df_hr_filter['hr_means'].replace(0.0, float('nan'))
#         df_hr_filter = df_hr_filter.interpolate(method='nearest').bfill()
        
        df_hr_filter = df_hr_filter.sort_values('from_time')
        
        convert_to_awd(df_hr_filter, 'hr_means', f'hr_means_channel_{channel}_{csv_file.split(".")[0]}.awd', data_meta)
        convert_to_awd(df_hr_filter, 'activity', f'activity_channel_{channel}_{csv_file.split(".")[0]}.awd', data_meta)
        
    return data_log
        


In [3]:
pilot_1 = generate_awd_from_calculated_nni_df('actogram_2_10min.csv', [5, 8], 5, 'Mouse-Pilot2')
pilot_2 = generate_awd_from_calculated_nni_df('actogram_1_10min.csv', [2], 5, 'Mouse-Pilot1')

/var/folders/zz/mk7_vrcn3r3gsbwr585w4blxzyk8xb/T/ipykernel_23176/1059794282.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hr_means'] = df['nni'].apply(lambda x: __to_hr_mean(x))


In [114]:
generate_awd_from_calculated_nni_df('actogram_2_5min.csv', [5, 8], 5, 'Mouse-Pilot2')
generate_awd_from_calculated_nni_df('actogram_1_5min.csv', [2], 5, 'Mouse-Pilot1')

/var/folders/zz/mk7_vrcn3r3gsbwr585w4blxzyk8xb/T/ipykernel_22480/3208912026.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hr_means'] = df['nni'].apply(lambda x: __to_hr_mean(x))
/var/folders/zz/mk7_vrcn3r3gsbwr585w4blxzyk8xb/T/ipykernel_22480/3208912026.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hr_means'] = df['nni'].apply(lambda x: __to_hr_mean(x))


In [75]:

for dt in set([s[0] for s in d.from_time.str.split(' ')]):
    print(dt, len(d[(d['from_time'].str.contains(dt)) & (d['channel']==5)]))

2023-04-29 169
2023-05-26 288
2023-06-15 288
2023-03-25 288
2023-04-02 288
2023-07-05 287
2023-07-11 288
2023-03-30 288
2023-04-03 288
2023-07-01 288
2023-03-31 288
2023-05-24 288
2023-07-15 288
2023-07-09 288
2023-04-16 288
2023-05-04 288
2023-05-27 288
2023-07-06 288
2023-05-16 288
2023-03-22 288
2023-04-14 288
2023-04-25 288
2023-05-31 288
2023-03-21 288
2023-03-26 288
2023-04-23 288
2023-05-05 288
2023-06-11 288
2023-07-02 288
2023-06-07 287
2023-03-17 288
2023-06-03 288
2023-06-26 288
2023-05-15 288
2023-06-20 288
2023-05-22 287
2023-04-04 288
2023-03-20 288
2023-06-04 288
2023-06-17 288
2023-07-04 288
2023-05-20 288
2023-06-05 288
2023-05-14 288
2023-04-08 288
2023-05-17 288
2023-06-21 288
2023-04-19 287
2023-04-30 288
2023-05-13 54
2023-06-28 288
2023-05-12 109
2023-04-15 288
2023-07-12 288
2023-06-14 288
2023-03-16 288
2023-06-30 288
2023-07-16 288
2023-03-24 288
2023-03-27 288
2023-05-30 288
2023-06-13 288
2023-06-19 288
2023-06-27 288
2023-06-23 288
2023-04-09 288
2023-06-10 

In [259]:
def identify_missing(csv_file, duration):
    log_data = {'channel':[], 'date':[], 'total data':[], 'status data':[], 'missing':[]}
    records_per_day  = int(60 * 24 / duration)
    df = load_csv_file(csv_file)
    channels = set(df['channel'])
    
    for channel in channels:
        dfr = df[df.channel==channel].reset_index(drop=True)

        start_date = datetime.combine(min(dfr.from_time).date(), datetime.min.time())
        finish_date = datetime.combine(max(dfr.from_time).date(), datetime.min.time())
        total_days = (finish_date - start_date).days + 1

        for day_num in range(total_days):
            current_date = start_date + timedelta(days=day_num)
            next_date = current_date + timedelta(days=1)
            df_records = dfr[dfr.from_time.gt(current_date) & dfr.from_time.le(next_date)].sort_values(by='from_time').reset_index(drop=True)
            log_data['channel'] += [channel]
            log_data['date'] += [current_date.strftime('%d-%m-%Y')]
            log_data['total data'] += [df_records.shape[0]]
            
            if current_date in [start_date, finish_date]:
                log_data['status data'] += ['begining/ending date']
                log_data['missing'] += ['']
            else: # need to check
                if df_records.shape[0] > records_per_day:
                    log_data['status data'] += ['containing duplicate/s']
                    log_data['missing'] += ['']
                elif df_records.shape[0] < records_per_day:
                    missing_loc = []
                    if df_records.shape[0] >= records_per_day-5:
                        log_data['status data'] += ['missing 1-5 records']
                        log_data['missing'] += ['']
                        continue

                    if df_records.shape[0] <= 1:
                        log_data['status data'] += ['missing all/mostly']
                        log_data['missing'] += ['']
                        continue

                    if (df_records.iloc[0]['from_time']-datetime.fromisoformat(current_date.isoformat())).total_seconds()/60 > duration:
                        t1 = current_date.strftime('%d-%m-%Y %H:%M:%S')
                        t2 = df_records.iloc[0]['from_time'].strftime('%d-%m-%Y %H:%M:%S')
                        missing_loc += [f"{t1} to {t2}"]
                    
                    for i in df_records.index:
                        if i == 0: continue
                        if (df_records.iloc[i]['from_time']-df_records.iloc[i-1]['from_time']).total_seconds()/60 > duration:
                            t1 = df_records.iloc[i-1]['from_time'].strftime('%d-%m-%Y %H:%M:%S')
                            t2 = df_records.iloc[i]['from_time'].strftime('%d-%m-%Y %H:%M:%S')
                            missing_loc += [f"{t1} to {t2}"]
                    
                    if (datetime.fromisoformat(next_date.isoformat()) - df_records.iloc[0]['from_time']).total_seconds()/60 > duration:
                        t1 = df_records.iloc[0]['from_time'].strftime('%d-%m-%Y %H:%M:%S')
                        t2 = next_date.strftime('%d-%m-%Y %H:%M:%S')
                        missing_loc += [f"{t1} to {t2}"]
                    
                    log_data['status data'] += ['missing']
                    log_data['missing'] += ['\n'.join(missing_loc)]
                else:
                    log_data['status data'] += ['complete']
                    log_data['missing'] += ['']
        
    return pd.DataFrame(log_data)

In [260]:
df_report1 = identify_missing('actogram_1_10min.csv', 5)

In [261]:
df_report2 = identify_missing('actogram_2_10min.csv', 5)

In [262]:
df1 = df_report1[(df_report1.channel==2) & (~(df_report1['status data']=='complete'))]

In [263]:
df2 = df_report2[(df_report2.channel.isin([5,8])) & (~(df_report2['status data']=='complete'))]

In [264]:
pd.concat([df1, df2], ignore_index=True)

,channel,date,total data,status data,missing
0,2,31-10-2022,162,begining/ending date,
1,2,01-11-2022,126,missing,01-11-2022 00:01:47 to 02-11-2022 00:00:00
2,2,02-11-2022,0,missing all/mostly,
3,2,03-11-2022,162,missing,03-11-2022 00:00:00 to 03-11-2022 10:32:12\n03...
4,2,22-11-2022,287,missing 1-5 records,
5,2,28-12-2022,287,missing 1-5 records,
6,2,01-02-2023,287,missing 1-5 records,
7,2,20-02-2023,273,begining/ending date,
8,8,15-03-2023,193,begining/ending date,
9,8,29-03-2023,287,missing 1-5 records,


In [269]:
df_report2.iloc[55]

channel                 8
date           09-05-2023
total data            288
status data      complete
missing                  
Name: 55, dtype: object